In [9]:
import pandas as pd
import pyodbc
import sqlite3

# iedereen heeft andere servername btw

In [10]:
servername = 'LAPTOP-RPEMJO16\\SQLEXPRESS'

# Northwind connectie

In [18]:
northwind_conn = pyodbc.connect('DRIVER={SQL SERVER};SERVER='+servername+';DATABASE=NorthWind; Trusted_Connection=yes')
nwCursor = northwind_conn.cursor()
nwCursor

# Adventureworks connectie

In [15]:
adventureworks_conn = pyodbc.connect('DRIVER={SQL SERVER};SERVER='+servername+';DATABASE=AdventureWorks2019; Trusted_Connection=yes')
awCursor = adventureworks_conn.cursor()
awCursor

# Aenc connectie

In [17]:
aenc_conn = pyodbc.connect('DRIVER={Microsoft Access Driver (*.mdb, *.accdb)};DBQ=../raw-databases/aenc.accdb;')
aencCursor = aenc_conn.cursor()

table_names = [table_info.table_name for table_info in aencCursor.tables()]

# Print table names
print(table_names)

['MSysAccessStorage', 'MSysACEs', 'MSysComplexColumns', 'MSysIMEXColumns', 'MSysIMEXSpecs', 'MSysNameMap', 'MSysNavPaneGroupCategories', 'MSysNavPaneGroups', 'MSysNavPaneGroupToObjects', 'MSysNavPaneObjectIDs', 'MSysObjects', 'MSysQueries', 'MSysRelationships', 'MSysResources', 'bonus', 'customer', 'department', 'employee', 'product', 'region', 'sales_order', 'sales_order_item', 'state']


In [21]:
aenctest = pd.read_sql_query('SELECT * FROM customer', aenc_conn)
awtest = pd.read_sql_query('SELECT * FROM Person.Address', adventureworks_conn)
nwtest = pd.read_sql_query('SELECT * FROM customers', northwind_conn)

# print(aenctest)
#print(awtest)
#print(nwtest)

IndentationError: unexpected indent (2368234722.py, line 6)

In [50]:
# Klopt echt helemaal niets van de aenc - > nw orders draw io

# Select NorthWind
nw_Emp = pd.read_sql_query('SELECT * FROM Employees', northwind_conn)
nw_Ord = pd.read_sql_query('SELECT * FROM Orders', northwind_conn)
nw_OrdDet = pd.read_sql_query('SELECT * FROM [Order Details]', northwind_conn)
# Select Production
Prod_Prod = pd.read_sql_query('SELECT * FROM Production.Product', adventureworks_conn)
Prod_Cat = pd.read_sql_query('SELECT * FROM Production.ProductCategory', adventureworks_conn)
# Select aenc
AE_Cus = pd.read_sql_query('SELECT * FROM customer', aenc_conn)
nw_Ord.head()
# aenc customer + nw employee ->    nw orders
MergeNwOrders1 = AE_Cus.merge(nw_Ord, left_on='id', right_on='CustomerID', how='right')
MergeNwOrders2 = nw_Emp.merge(MergeNwOrders1, on='EmployeeID', how='right')
#

MergedProdProd = Prod_Cat.merge(Prod_Prod, left_on='ProductCategoryID', right_on='ProductSubcategoryID', how='right')

NWorderDetails1 = MergeNwOrders2.merge(nw_OrdDet, on='OrderID', how='right')
NWorderDetailsFinal = MergedProdProd.merge(NWorderDetails1, on='ProductID', how='right')

C:\Users\kwand\AppData\Local\Temp\ipykernel_6788\2828923582.py:4: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  nw_Emp = pd.read_sql_query('SELECT * FROM Employees', northwind_conn)
C:\Users\kwand\AppData\Local\Temp\ipykernel_6788\2828923582.py:5: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  nw_Ord = pd.read_sql_query('SELECT * FROM Orders', northwind_conn)
C:\Users\kwand\AppData\Local\Temp\ipykernel_6788\2828923582.py:6: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  nw_OrdDet = pd.read_sql_query('SELECT * FROM [Order Deta

,ProductCategoryID,Name_x,rowguid_x,ModifiedDate_x,ProductID,Name_y,ProductNumber,MakeFlag,FinishedGoodsFlag,Color,...,Freight,ShipName,ShipAddress,ShipCity,ShipRegion,ShipPostalCode,ShipCountry,UnitPrice,Quantity,Discount
0,NaN,NaN,NaN,NaT,11,NaN,NaN,NaN,NaN,NaN,...,32.38,Vins et alcools Chevalier,59 rue de l'Abbaye,Reims,None,51100,France,14.00,12,0.00
1,NaN,NaN,NaN,NaT,42,NaN,NaN,NaN,NaN,NaN,...,32.38,Vins et alcools Chevalier,59 rue de l'Abbaye,Reims,None,51100,France,9.80,10,0.00
2,NaN,NaN,NaN,NaT,72,NaN,NaN,NaN,NaN,NaN,...,32.38,Vins et alcools Chevalier,59 rue de l'Abbaye,Reims,None,51100,France,34.80,5,0.00
3,NaN,NaN,NaN,NaT,14,NaN,NaN,NaN,NaN,NaN,...,11.61,Toms Spezialitäten,Luisenstr. 48,Münster,None,44087,Germany,18.60,9,0.00
4,NaN,NaN,NaN,NaT,51,NaN,NaN,NaN,NaN,NaN,...,11.61,Toms Spezialitäten,Luisenstr. 48,Münster,None,44087,Germany,42.40,40,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2150,NaN,NaN,NaN,NaT,64,NaN,NaN,NaN,NaN,NaN,...,8.53,Rattlesnake Canyon Grocery,2817 Milton Dr.,Albuquerque,NM,87110,USA,33.25,2,0.03
2151,NaN,NaN,NaN,NaT,66,NaN,NaN,NaN,NaN,NaN,...,8.53,Rattlesnake Canyon Grocery,2817 Milton Dr.,Albuquerque,NM,87110,USA,17.00,1,0.00
2152,NaN,NaN,NaN,NaT,73,NaN,NaN,NaN,NaN,NaN,...,8.53,Rattlesnake Canyon Grocery,2817 Milton Dr.,Albuquerque,NM,87110,USA,15.00,2,0.01
2153,NaN,NaN,NaN,NaT,75,NaN,NaN,NaN,NaN,NaN,...,8.53,Rattlesnake Canyon Grocery,2817 Milton Dr.,Albuquerque,NM,87110,USA,7.75,4,0.00
